**Disclaimer:** The distributed notebook only serves for demonstration purposes. Jupyter notebooks are not recommended for preCICE development or deployment. Feel free to convert this notebook to a plain python script for improved error messages using `jupyter nbconvert --to python Solver*.ipynb`

In [1]:
def dummySolidSolver(x, y, dt):
    return x + dt * y**2

In [2]:
import precice
#precice_config = "precice-config-both-init.xml"
precice_config = "precice-config-experimental-both-init.xml"
solverInterface = precice.Interface("Solid", precice_config, 0,1)
meshId = solverInterface.get_mesh_id("Solid-Mesh")
vertexId = solverInterface.set_mesh_vertex(meshId, [1,2,3])
readDataId = solverInterface.get_data_id("Force", meshId)
writeDataId = solverInterface.get_data_id("Displacement", meshId)

(0) 18:10:44 [config::ParticipantConfiguration]:352 in xmlTagCallback: WARNING: You configured the read data with name "dataOne" to use the waveform-order="1", which is currently still experimental. Use with care.
(0) 18:10:44 [impl::SolverInterfaceImpl]:169 in configure: This is preCICE version 2.3.0
(0) 18:10:44 [impl::SolverInterfaceImpl]:170 in configure: Revision info: v2.3.0-120-g5b868a24
(0) 18:10:44 [impl::SolverInterfaceImpl]:172 in configure: Configuration: Debug
(0) 18:10:44 [impl::SolverInterfaceImpl]:176 in configure: Configuring preCICE with configuration "precice-config-experimental-both-init.xml"
(0) 18:10:44 [impl::SolverInterfaceImpl]:177 in configure: I am participant "SolverTwo"


In [3]:
precice_dt = solverInterface.initialize()

(0) 18:10:44 [impl::SolverInterfaceImpl]:250 in initialize: Setting up master communication to coupling partner/s
(0) 18:10:48 [impl::SolverInterfaceImpl]:264 in initialize: Masters are connected
(0) 18:10:48 [impl::SolverInterfaceImpl]:268 in initialize: Setting up preliminary slaves communication to coupling partner/s
(0) 18:10:48 [partition::ReceivedPartition]:69 in communicate: Receive global mesh MeshOne
(0) 18:10:48 [partition::ProvidedPartition]:121 in prepare: Prepare partition for mesh MeshTwo
(0) 18:10:48 [impl::SolverInterfaceImpl]:276 in initialize: Setting up slaves communication to coupling partner/s
(0) 18:10:48 [impl::SolverInterfaceImpl]:282 in initialize: Slaves are connected
(0) 18:10:48 [impl::SolverInterfaceImpl]:316 in initialize: iteration: 1 of 2, time-window: 1 of 2, time: 0, time-window-size: 1, max-timestep-length: 1, ongoing: yes, time-window-complete: no, write-initial-data write-iteration-checkpoint 


In [4]:
window_size = precice_dt
solver_dt = window_size/3
time = 0

### Initialize data
optional, if `initialize = "false"` or not given

In [5]:
initDisplacement = 2
writeData = initDisplacement
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)

In [6]:
solverInterface.mark_action_fulfilled(precice.action_write_initial_data())

In [7]:
solverInterface.initialize_data()

(0) 18:10:48 [impl::SolverInterfaceImpl]:1673 in computeMappings: Compute "write" mapping from mesh "MeshTwo" to mesh "MeshOne".
(0) 18:10:48 [mapping::NearestNeighborMapping]:89 in computeMapping: Mapping distance min:0 max:0 avg: 0 var: 0 cnt: 1
(0) 18:10:48 [impl::SolverInterfaceImpl]:1673 in computeMappings: Compute "read" mapping from mesh "MeshOne" to mesh "MeshTwo".
(0) 18:10:48 [mapping::NearestNeighborMapping]:89 in computeMapping: Mapping distance min:0 max:0 avg: 0 var: 0 cnt: 1


### First window, first iteration

require to write checkpoint, when entering window:

In [8]:
solverInterface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())

#### First subcycle

In [9]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))

readData = 1.0


In [10]:
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

Start time: 0
End time: 0.3333333333333333
writeData = 2.3333333333333335
precice_dt = 0.6666666666666667
(0) 18:10:48 [impl::SolverInterfaceImpl]:456 in advance: iteration: 1 of 2, time-window: 1 of 2, time: 0.333333, time-window-size: 1, max-timestep-length: 0.666667, ongoing: yes, time-window-complete: no, 


#### Second subcycle

In [11]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

readData = 1.0
Start time: 0.3333333333333333
End time: 0.6666666666666666
writeData = 2.666666666666667
precice_dt = 0.33333333333333337
(0) 18:10:48 [impl::SolverInterfaceImpl]:456 in advance: iteration: 1 of 2, time-window: 1 of 2, time: 0.666667, time-window-size: 1, max-timestep-length: 0.333333, ongoing: yes, time-window-complete: no, 


#### Third subcycle

In [12]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

readData = 1.0
Start time: 0.6666666666666666
End time: 1.0
writeData = 3.0000000000000004
precice_dt = 1.0(0) 18:10:48 [cplscheme::BaseCouplingScheme]:542 in measureConvergence: min iteration convergence measure: #it = 1 of 5, conv = false
(0) 18:10:48 [impl::SolverInterfaceImpl]:456 in advance: iteration: 2 of 2, time-window: 1 of 2, time: 0, time-window-size: 1, max-timestep-length: 1, ongoing: yes, time-window-complete: no, read-iteration-checkpoint 



In [13]:
solverInterface.is_time_window_complete()

False

### First window, second iteration
require to read checkpoint, when repeating window:

In [14]:
solverInterface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())

#### First subcycle

In [15]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))

readData = 0.3333333333333334


In [16]:
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))

Start time: 1.0
End time: 1.3333333333333333


In [17]:
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)

writeData = 3.0370370370370376


In [18]:
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

precice_dt = 0.6666666666666667
(0) 18:10:48 [impl::SolverInterfaceImpl]:456 in advance: iteration: 2 of 2, time-window: 1 of 2, time: 0.333333, time-window-size: 1, max-timestep-length: 0.666667, ongoing: yes, time-window-complete: no, 


#### Second subcycle

In [19]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

readData = -0.33333333333333304
Start time: 1.3333333333333333
End time: 1.6666666666666665
writeData = 3.0740740740740744
precice_dt = 0.33333333333333337(0) 18:10:48 [impl::SolverInterfaceImpl]:456 in advance: iteration: 2 of 2, time-window: 1 of 2, time: 0.666667, time-window-size: 1, max-timestep-length: 0.333333, ongoing: yes, time-window-complete: no, 



#### Third subcycle

In [20]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

readData = -1.0
Start time: 1.6666666666666665
End time: 1.9999999999999998
writeData = 3.407407407407408
precice_dt = 1.0(0) 18:10:49 [cplscheme::BaseCouplingScheme]:542 in measureConvergence: min iteration convergence measure: #it = 2 of 5, conv = false
(0) 18:10:49 [cplscheme::BaseCouplingScheme]:231 in advance: Time window completed
(0) 18:10:49 [impl::SolverInterfaceImpl]:456 in advance: iteration: 1 of 2, time-window: 2 of 2, time: 1, time-window-size: 1, max-timestep-length: 1, ongoing: yes, time-window-complete: yes, write-iteration-checkpoint 



In [21]:
solverInterface.is_time_window_complete()

True

### Second window:
... same procedure as before.

In [22]:
window = 2
iteration = 1

while solverInterface.is_coupling_ongoing():
    if solverInterface.is_action_required(precice.action_write_iteration_checkpoint()):
        solverInterface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())
        
    readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
    print("readData = {}".format(readData))
    
    writeData = dummySolidSolver(writeData, readData, solver_dt)
    
    print("writeData = {}".format(writeData))
    solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
    
    precice_dt = solverInterface.advance(solver_dt)
    print("precice_dt = {}".format(precice_dt))
    
    if solverInterface.is_time_window_complete():
        window += 1
        iteration = 1
    if solverInterface.is_action_required(precice.action_read_iteration_checkpoint()):
        iteration += 1
    
    if solverInterface.is_coupling_ongoing():
        print("window = {}".format(window))
        print("iteration = {}".format(iteration))
        
    if solverInterface.is_action_required(precice.action_read_iteration_checkpoint()):
        solverInterface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())
        
print("Done!")
solverInterface.finalize()

readData = -2.0
writeData = 4.740740740740741
precice_dt = 0.6666666666666667
(0) 18:10:49 [impl::SolverInterfaceImpl]:456 in advance: iteration: 1 of 2, time-window: 2 of 2, time: 1.33333, time-window-size: 1, max-timestep-length: 0.666667, ongoing: yes, time-window-complete: no, 
window = 2
iteration = 1
readData = -2.9999999999999996
writeData = 7.7407407407407405
precice_dt = 0.33333333333333337
window = 2
iteration = 1
readData = -4.0
writeData = 13.074074074074073
precice_dt = 1.0
window = 2
iteration = 2
readData = -3.135802469135803(0) 18:10:49 [impl::SolverInterfaceImpl]:456 in advance: iteration: 1 of 2, time-window: 2 of 2, time: 1.66667, time-window-size: 1, max-timestep-length: 0.333333, ongoing: yes, time-window-complete: no, 
(0) 18:10:49 [cplscheme::BaseCouplingScheme]:542 in measureConvergence: min iteration convergence measure: #it = 1 of 5, conv = false
(0) 18:10:49 [impl::SolverInterfaceImpl]:456 in advance: iteration: 2 of 2, time-window: 2 of 2, time: 1, time-wind